In [2]:
##Qdrant
## importando o client
from qdrant_client import QdrantClient
## importando módulos que ajudam a criar nossa coleção
from qdrant_client.http.models import Distance, VectorParams

##langchain
## importando o Qdrant como vector store
# from langchain.vectorstores import Qdrant
from langchain_qdrant import Qdrant
## importando o OpenAi embeddings
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
## Importando função que auxilia na quebra de textos em chunks
from langchain.text_splitter import CharacterTextSplitter
## importando o módulo que facilita o uso de vector stores em QA(question answering)
from langchain.chains import RetrievalQA
## importando a llm
from langchain_openai import OpenAI

import os
from dotenv import load_dotenv

In [3]:
## carregando as variaveis de ambiente
load_dotenv()

True

In [4]:
## instanciando nosso client do Qdrant
client = QdrantClient(host="localhost", port=6333)

In [5]:
## usando o client para verificar se a collection já existe, caso contrário ele irá criar uma
collection_exists = client.collection_exists(collection_name="documentos_collection")
if collection_exists:
    print("A collection já existe!")
else:
    ## agora vamos criar a nossa coleção
    client.create_collection(
        collection_name="documentos_collection",
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
    )


A collection já existe!


In [6]:
## vamos instanciar o OpenAiEmbeddings() lembrado que já temos a nossa variável de ambiente com a chave de api na nomenclatura padrão

embeddings = OpenAIEmbeddings()
vector_store = Qdrant(
    client=client,
    collection_name="documentos_collection",
    embeddings=embeddings
)

In [7]:
### criando a função que quebra nosso arquivo de texto em chunks
def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    chunks = text_splitter.split_text(text)
    return chunks

## abrindo e lendo o nosso arquivo
with open("../data/base_dados.txt") as file:
    raw_text = file.read()

texts = get_chunks(text=raw_text)
vector_store.add_texts(texts)

['ee0f7ee0312d4ee5b60c4d4198afbd86',
 '4f072a47dd0e4278bd37f5bfad204fb9',
 '59702d90948b44ebb7db144275d5738d',
 'c5e0a78c88eb4fa69b78d0c3bc0b486c',
 '21e9f8f2b81e44c299e9eddb8a0665a2',
 '3c8860e049a1434ba3a62ea37c1630bb',
 'fa8abcdd7e9a4f429fb8c6dfb2ea73b9',
 'ee482d20103c4e8abbeaa46a260883a5',
 'e1ec945f3fc8439492493cc81a643798',
 '7a39fc2207d548f78a238e0e32529857',
 'd14c17b926164dc2b3c932bfd2e07310',
 '9bf28fa0eac44e5881eea8fa6066dda4',
 'e79cb12dfdbb42788dde0a771f2cc07e',
 '0795396c8b1f4e89a64e95963636e3fb',
 '5ddebb7ad6c745f3990af90b284be2d2',
 'e7858f9ece014610b81803cbbbfd21ca',
 '508dde3003d84ca58e3e29f1a51883b9',
 'dae97076a0ce4d2b8364eea397f302ef',
 'dee0bbd52d2144a6bd3abf0149c60c98',
 '9aa9d9afcbfd4ca393841d1f81f01c9c',
 '8af29c0dc82445fbae364351f9461329',
 'ec1b9d04b4cd43198d1d48c9e2ddc2dc',
 'b77537f2cae44c6faf1207b725bd411e',
 '14747b16f9c84807b086629dbdcf3854',
 '549899c63fbe411fb8cb8904d3052918',
 '9ed18260260047fe8a23ffbfcf92af69',
 'bfa35b1ecab241ea98a47f940aadac14',
 

In [8]:
### montando a arquitetura de recebimento da pergunta, transformação em embedding, busca por similaridade no nosso vector database
embeddings = OpenAIEmbeddings()
vector_store = Qdrant(
    client=client,
    collection_name="documentos_collection",
    embeddings=embeddings
)
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [9]:
question = "como funciona as comissões da hotmart?"
response = qa.invoke(question)

In [10]:
from pprint import pprint
print(response.get("result"))

 As comissões da Hotmart funcionam como uma taxa por venda realizada, sendo que a plataforma recebe uma porcentagem do preço do produto e um valor fixo por venda. A taxa pode variar dependendo do preço do produto e da moeda em que a venda foi realizada, e é paga somente quando uma venda é concretizada.
